# *Jared Galloway*

*Introduction and TreeSequence Primer*

*Matsen Lab Meeting - May 15, 2020*

# *Elevator Introduction*

 
* Undergraduate in Computer and Information Science, University of Oregon.

 * Started research surrounding population genetics slightly over 3 years ago w/ Peter Ralph IE$^{2}$

 * Took a gap year to continue research and learn more about ML/DL with Peter and Andy.

* Started a Masters in Biology at the University of Oregon

* Now, here I am!

# *The fun stuff: TreeSequence Primer*

For a set of sampled chromosomes, at each position along the genome there is a genealogical tree that says how they are related. 

The tabular data structure known as a `TreeSequence`, created by Jerome Kelleher at Oxford Big Data Institute, allows us to effeciently store these related histories and query them with a poerful, community maintined, set of tools included in
`tskit`

![SegmentLocal](./materials/sim_ts.anim.gif "segment")

## *Let's back up a little ... Genomes and Genealogies*

### Genomes

* reflect The basic mechanisms of life!

* Are _very_ big, from $\approx 10^{7}$ to $10^{12}$ nucleotides long

* With Next Generation Sequencing, we are able to sequence _lots_ of samples.

* They are the result of the population process throughout history...

### Genealogies

* a single genealogy describes the history resulting in an individual's genome.

<img src="materials/family_tree.png" width="750" align="center">

[gcbias](https://gcbias.org/2013/11/11/how-does-your-number-of-genetic-ancestors-grow-back-over-time/)

* We are all related by an _expansive_ genealogical network

* We can only observe _genomes_ from recent history (with some exceptions), that have been derived from the stochastic processes of mutation and recombination throughout history. 

 * What can we learn about the structure, history, and biology of a _genome_?

## Modeling

* Without recombination, there would be a **single** coalescent tree describing this relationship across the genome for **all** sampled genomes.

* We can study the the genealogy of a single locus (w/ no recombination) using the Kingman coalescent model

<img src="materials/coalecent.png" width="750" align="center">

TODO: Cite

## Meiosis & Recombination

* Recombination splits the ancestry on either side of the chromosome at the point where it occurs.

* This meant that a more complex model was needed if we wanted to study linkage between any loci passed down through generations, split by recombination.

* Soon after the single locus coalescent was derived, Hudson defined an algorithm to simulate the coalescent with recombination

* These models ended up resulting what's known as the *Ancestral Recombination Graph*

### Ancestral Recombination Graph (ARG)

* In the ARG, nodes are events (either recombination or common ancestor) and the edges are ancestral chromosomes. 

* A recombination event results in a single ancestral chromosome splitting into two chromosomes, and a common ancestor event results in two chromosomes merging into a common ancestor

### The problem

* The ARG has a scaling problem with number of reccombination events

* This is because essentially with each recombination event, ancestral trees split and all the information in the tree had to be copied over to a new tree.

* seems to imply that simulating the coalescent with recombination over chromosome scales is hopeless :(

* But wait!

[Kelleher, Etheridge, & McVean 2016](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4856371/)

msprime: whole genomes with $N \approx 10^{5}$ in O(minutes)

<img src="materials/msprime-time.png" width="750" align="center">

## `TreeSequence` Tabular Data Structure

* Key idea, marginal trees along the genome are _highly_ correlated with neighboring.

* Storing a tree sequence in the four tables - nodes, edges, sites, and mutations - is succinct (no redundancy).

<img src="materials/example_tree_sequence.png" width="750" align="center">

## `msprime` and `tskit` python API

In [36]:
import msprime
ts = msprime.simulate(5, recombination_rate=0)
print(ts.first().draw(format="unicode"))

    8    
 ┏━━┻━┓  
 ┃    7  
 ┃  ┏━┻┓ 
 6  ┃  ┃ 
┏┻┓ ┃  ┃ 
┃ ┃ ┃  5 
┃ ┃ ┃ ┏┻┓
0 4 3 1 2



In [40]:
print(f"Node Table:\n\n {ts.tables.nodes}")

Node Generated:

 id	flags	population	individual	time	metadata
0	1	0	-1	0.00000000000000	
1	1	0	-1	0.00000000000000	
2	1	0	-1	0.00000000000000	
3	1	0	-1	0.00000000000000	
4	1	0	-1	0.00000000000000	
5	0	0	-1	0.29044375220648	
6	0	0	-1	0.33685990414120	
7	0	0	-1	1.74497258380198	
8	0	0	-1	4.48231130730983	


In [41]:
print(f"Edge Table:\n\n {ts.tables.edges}")

Node Generated:

 id	left		right		parent	child
0	0.00000000	1.00000000	5	1
1	0.00000000	1.00000000	5	2
2	0.00000000	1.00000000	6	0
3	0.00000000	1.00000000	6	4
4	0.00000000	1.00000000	7	3
5	0.00000000	1.00000000	7	5
6	0.00000000	1.00000000	8	6
7	0.00000000	1.00000000	8	7


A `TreeSequence` Object contains a collection of `Tree` Objects

In [44]:
print(type(ts))
print(type(ts.trees()))
marginal_tree_iter = ts.trees()
print(type(next(marginal_tree_iter)))
print(f"There are {ts.num_trees} marginal trees in this tree sequence")
# print(type(next(marginal_tree_iter)))

<class 'tskit.trees.TreeSequence'>
<class 'tskit.trees.TreeIterator'>
<class 'tskit.trees.Tree'>
There are 1 marginal trees in this tree sequence


1. how did the tree sequence data structure improve things
2. how does it work.
3. Examples
4. Cool, we've seen how this scales 

## Forward Time Simulation - `SLiM`

1. why are fm more powerful that
2. recording tree sequences
3. simplification
4. examples and results - pyslim

## Other Projects

* Stickleback Simulations
* ReLERNN
* StdPopSim


## More people to thank

* Peter Ralph
* Andrew Kern
* Leslie Coonrod
* Bill Cresko
* Stacey Wagner
* Ben Haller
* Jerome Kelleher
* Clay Small
